In [ ]:
import sys
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install torch torchvision transformers

In [ ]:
!pip install emoji

In [ ]:
train_df = pd.read_csv('train.csv')
dev_df = pd.read_csv('dev.csv')
test_df = pd.read_csv('test.csv')

In [ ]:
train_df

,Unnamed: 0,text,label,id
0,0,4. Can eating garlic help prevent infection wi...,0,1250219300389974016
1,1,French police chief killed himself after #Char...,1,554886875303780352
2,2,Coronavirus disease (COVID-19) advice for the ...,0,1237901309011021825
3,3,Ottawa police confirm that there were multiple...,0,524958128392376320
4,4,if the primary focus of a government isn't to ...,0,1239295488677085185
...,...,...,...,...
1802,1802,Desperate Ted Cruz Claims Planned Parenthood S...,1,671181758692507648
1803,1803,"""Thoughts and prayers are not enough."" Pres. O...",1,672513234419638273
1804,1804,Police have surrounded this building where the...,0,553508098825261056
1805,1805,@Kirstenjoyweiss @MattFabrication @prestone85 ...,0,1249479605582327808


In [ ]:
from transformers import BertTokenizer
import tensorflow as tf 
from tensorflow.keras.optimizers import Adam
from transformers import TFBertModel
from keras import backend as k_backend

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
def bert_encode(data, max_seq_length) :
    input_ids = []
    attention_masks = []
    for i in range(len(data.text)):
        encoded = tokenizer.encode_plus(
            data.text[i],
            add_special_tokens=True,
            max_length=max_seq_length,
            pad_to_max_length=True,
            return_attention_mask=True,
            truncation = True
        )

        input_ids.append(encoded['input_ids'])
        attention_masks.append(encoded['attention_mask'])
    return np.array(input_ids),np.array(attention_masks)

In [ ]:
def recall(y_true, y_pred):
    true_positives = k_backend.sum(k_backend.round(k_backend.clip(y_true * y_pred, 0, 1)))
    possible_positives = k_backend.sum(k_backend.round(k_backend.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + k_backend.epsilon())
    return recall

def precision(y_true, y_pred):
    true_positives = k_backend.sum(k_backend.round(k_backend.clip(y_true * y_pred, 0, 1)))
    predicted_positives = k_backend.sum(k_backend.round(k_backend.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + k_backend.epsilon())
    return precision
    
def f1(y_true, y_pred):
    precision_value = precision(y_true, y_pred)
    recall_value = recall(y_true, y_pred)
    return 2*((precision_value*recall_value) / (precision_value+recall_value+k_backend.epsilon()))

In [ ]:
def create_model(bert_model):
    input_ids = tf.keras.Input(shape=(300,),dtype='int32')
    attention_masks = tf.keras.Input(shape=(300,),dtype='int32')

    output = bert_model([input_ids,attention_masks])
    output = output[1]
    output = tf.keras.layers.Dense(32,activation='relu')(output)
    output = tf.keras.layers.Dropout(0.3)(output)
    output = tf.keras.layers.Dense(1,activation='sigmoid')(output)
    model = tf.keras.models.Model(inputs = [input_ids,attention_masks],outputs = output)
    model.compile(Adam(lr=6e-6), loss='binary_crossentropy', metrics=['accuracy', recall, precision, f1])
    return model

In [ ]:
TFbert_model = TFBertModel.from_pretrained('bert-base-uncased')
model = create_model(TFbert_model)

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
train_input_ids,train_attention_masks = bert_encode(train_df,300)
dev_input_ids,dev_attention_masks = bert_encode(dev_df,300)
test_input_ids,test_attention_masks = bert_encode(test_df,300)
dev_data = ([dev_input_ids, dev_attention_masks], dev_df.label)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2269: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
history = model.fit([train_input_ids,train_attention_masks], train_df.label, validation_data=dev_data ,callbacks=[callback], epochs=8, batch_size=4)

Epoch 1/8
452/452 [==============================] - 167s 369ms/step - loss: 0.4593 - accuracy: 0.7825 - recall: 0.1654 - precision: 0.1742 - f1: 0.1603 - val_loss: 0.3603 - val_accuracy: 0.8252 - val_recall: 0.2707 - val_precision: 0.2819 - val_f1: 0.2707
Epoch 2/8
452/452 [==============================] - 163s 362ms/step - loss: 0.2573 - accuracy: 0.9009 - recall: 0.4991 - precision: 0.4851 - f1: 0.4810 - val_loss: 0.2984 - val_accuracy: 0.8706 - val_recall: 0.3647 - val_precision: 0.4161 - val_f1: 0.3781
Epoch 3/8
452/452 [==============================] - 163s 362ms/step - loss: 0.1494 - accuracy: 0.9546 - recall: 0.5996 - precision: 0.5931 - f1: 0.5909 - val_loss: 0.2660 - val_accuracy: 0.9076 - val_recall: 0.4866 - val_precision: 0.5067 - val_f1: 0.4879
Epoch 4/8
452/452 [==============================] - 163s 361ms/step - loss: 0.0941 - accuracy: 0.9745 - recall: 0.6180 - precision: 0.6114 - f1: 0.6112 - val_loss: 0.2359 - val_accuracy: 0.9143 - val_recall: 0.4720 - val_precisi

In [ ]:
model.save(".TFbert_final")

INFO:tensorflow:Assets written to: .TFbert_final/assets


INFO:tensorflow:Assets written to: .TFbert_final/assets


In [ ]:
pre_dev_labels = model.predict([dev_input_ids, dev_attention_masks])
pre_dev_labels = np.round(pre_dev_labels).astype(int)

In [ ]:
import sklearn.metrics as metrics
print(metrics.accuracy_score(dev_df.label, pre_dev_labels))

0.9378151260504202


In [ ]:
print(metrics.precision_score(dev_df.label, pre_dev_labels))
print(metrics.recall_score(dev_df.label, pre_dev_labels))
print(metrics.f1_score(dev_df.label, pre_dev_labels, average='macro'))
print(metrics.roc_auc_score(dev_df.label, pre_dev_labels))

0.8623188405797102
0.8686131386861314
0.9125086934922999
0.913564211264463


In [ ]:
# For Kaggle
result = model.predict([test_input_ids, test_attention_masks])
result = np.round(result).astype(int)

In [ ]:
with open('test.predictions_TF.txt', 'w') as output:
  output.write('Id,Predicted\n')
  counter = 0
  for elem in list(result):
    for elem2 in list(elem):
      output.write(str(counter) + ',' + str(elem2)+'\n')
      counter+=1


In [ ]:
#For Task 2
covid_df = pd.read_csv('covid.csv')

In [ ]:
covid_input_ids,covid_attention_masks = bert_encode(covid_df,512)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2269: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
result_covid = model.predict([covid_input_ids, covid_attention_masks])
result_covid = np.round(result_covid).astype(int)

In [ ]:
with open('covid.predictions.txt', 'w') as output:
  output.write('Id,Predicted\n')
  counter = 0
  for elem in list(result_covid):
    for elem2 in list(elem):
      output.write(str(counter) + ',' + str(elem2)+'\n')
      counter+=1

In [ ]:
#covid_labels = np.argmax(result_covid, axis = -1)
#covid_output = pd.DataFrame({'id':covid_df.id,'target':covid_labels})
#covid_output.to_csv("covid_labels.csv",index=False,sep=',')